In [1]:
#import required libraries
import os
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
os.environ['TF_DETERMINISTIC_OPS'] = '1'
import cv2
import numpy as np
from imutils import paths
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import classification_report, roc_auc_score, matthews_corrcoef, precision_score, f1_score, recall_score
from xgboost import XGBClassifier
from sklearn.naive_bayes import GaussianNB
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, SimpleRNN,Dropout
from tensorflow.keras.utils import to_categorical
import pandas as pd
import time
import psutil
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications import VGG19
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input, decode_predictions
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
import keras

tf.keras.utils.set_random_seed(0)
tf.config.experimental.enable_op_determinism()

2024-03-27 18:06:57.060320: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-27 18:06:57.060403: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-27 18:06:57.063195: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-27 18:06:57.082754: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-27 18:06:59.647340: W ten

In [2]:
#!pip install tensorflow-tensorrt


In [3]:
#!pip install opencv-python
#!pip install imutils
#!pip install scikit-learn
#!pip install xgboost


In [4]:
#Class to load the dataset images from device

class SimpleDatasetLoader:
    # Method: Constructor
    def __init__(self, preprocessors=None):
        """
        :param preprocessors: List of image preprocessors
        """
        self.preprocessors = preprocessors

        if self.preprocessors is None:
            self.preprocessors = []

    # Method: Used to load a list of images for pre-processing
    def load(self, image_paths, verbose=-1):
        """
        :param image_paths: List of image paths
        :param verbose: Parameter for printing information to console
        :return: Tuple of data and labels
        """
        data, labels = [], []

        for i, image_path in enumerate(image_paths):
            image = cv2.imread(image_path)
            label = image_path.split(os.path.sep)[-2]

            if self.preprocessors is not None:
                for p in self.preprocessors:
                    image = p.preprocess(image)

            data.append(image)
            labels.append(label)

            if verbose > 0 and i > 0 and (i+1) % verbose == 0:
                print('[INFO]: Processed {}/{}'.format(i+1, len(image_paths)))

        return (np.array(data), np.array(labels))

In [5]:
#Class Preprocessror 
class SimplePreprocessor:
    # Method: Constructor
    def __init__(self, width, height, interpolation=cv2.INTER_AREA):
        """
        :param width: Image width
        :param height: Image height
        :param interpolation: Interpolation algorithm
        """
        self.width = width
        self.height = height
        self.interpolation = interpolation

    # Method: Used to resize the image to a fixed size (ignoring the aspect ratio)
    def preprocess(self, image):
        """
        :param image: Image
        :return: Re-sized image
        """
        return cv2.resize(image, (self.width, self.height), interpolation=self.interpolation)

In [6]:
from imutils import paths
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from __main__ import SimplePreprocessor
from __main__ import SimpleDatasetLoader

In [7]:
# Function to load and preprocess data using SimpleDatasetLoader
def load_and_preprocess_data(image_paths, target_size):
    sp = SimplePreprocessor(target_size[0], target_size[1])
    sdl = SimpleDatasetLoader(preprocessors=[sp])
    data, labels = sdl.load(image_paths, verbose=1000)

    print(data.shape)
    print(labels.shape)

    # Convert labels to one-hot encoding
    le = LabelEncoder()
    labels = le.fit_transform(labels)
    #labels = to_categorical(labels)

    return data, labels



# Get list of image paths
image_paths = list(paths.list_images("../BreaKHis_v1"))

# Define target size for images
target_size = (224, 224)  # Change this to your desired size

# Load and preprocess data
print('[INFO]: Images loading....')
data, labels = load_and_preprocess_data(image_paths, target_size)

[INFO]: Images loading....
[INFO]: Processed 1000/7909
[INFO]: Processed 2000/7909
[INFO]: Processed 3000/7909
[INFO]: Processed 4000/7909
[INFO]: Processed 5000/7909
[INFO]: Processed 6000/7909
[INFO]: Processed 7000/7909
(7909, 224, 224, 3)
(7909,)


In [8]:
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.models import Model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(target_size[0], target_size[1], 3))
model = Model(inputs=base_model.input, outputs=base_model.output)
# Extract features
features = model.predict(data, batch_size=32, verbose=1)
 
# Flatten the features
features_flatten = features.reshape(features.shape[0], -1)
    
print(features_flatten.shape)

2024-03-27 18:12:23.654970: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 37894 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-40GB, pci bus id: 0000:65:00.0, compute capability: 8.0
2024-03-27 18:12:32.830973: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
2024-03-27 18:12:33.176478: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


248/248 [==============================] - 7s 19ms/step
(7909, 25088)


In [9]:
data = features_flatten

In [10]:
import gc
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
#tf.config.experimental.set_memory_growth(tf.config.list_physical_devices('GPU')[0], True)
#tf.config.experimental.enable_tensor_float_32_execution(True)
#tf.config.threading.set_inter_op_parallelism_threads(1)
#tf.config.threading.set_intra_op_parallelism_threads(1)
# Initialize a DataFrame to store results
results_df = pd.DataFrame(columns=['Model', 'Average Accuracy', 'Average Sensitivity', 'Average Specificity',
                                   'Average AUC-ROC', 'Average MCC', 'Average Precision', 'Average F1 Score',
                                   'Memory Used (MB)', 'Time (s)'])

# Function to evaluate a model
def evaluate_model(model, name, data, labels):
    if name == "VGG16":
        labels = to_categorical(labels)

    elif name == "VGG19":  
        labels = to_categorical(labels)
    else:
        data = data
    accuracy_list = []
    sensitivity_list = []
    specificity_list = []
    auc_roc_list = []
    mcc_list = []
    precision_list = []
    f1_list = []
    time_start = time.time()
    memory_start = psutil.Process(os.getpid()).memory_info().rss / 1024 ** 2

    skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=0)
    if name == "VGG16" or name == "VGG19":


        with tf.device('/GPU:0'):
            tf.random.set_seed(0)
            for fold, (train_index, test_index) in enumerate(skf.split(data, labels.argmax(axis=1))):
                print(f'\n[INFO] Fold {fold + 1} / 10 for {name}')

                X_train, X_test = data[train_index], data[test_index]
                y_train, y_test = labels[train_index], labels[test_index]


                # Adding early stopping to prevent overfitting
                early_stopping = EarlyStopping(monitor='loss', patience=3, restore_best_weights=True)
                keras.utils.set_random_seed(0)
                np.random.seed(0)
                tf.random.set_seed(0)
                #tf.config.run_functions_eagerly(True)
                tf.data.experimental.enable_debug_mode()
                # Train the model
                # Example of using tf.data.Dataset
                train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
                train_dataset = train_dataset.shuffle(buffer_size=len(X_train)).batch(64)

                model.fit(train_dataset, epochs=20, callbacks=[early_stopping])

                # Evaluate the model
                predictions = model.predict(X_test)
                y_pred = np.argmax(predictions, axis=1)
                y_test_encoded = np.argmax(y_test, axis=1)

                
                # Calculate evaluation metrics for the current fold
                accuracy = np.mean(y_pred == y_test_encoded)
                sensitivity = recall_score(y_test_encoded, y_pred, pos_label=1)
                specificity = recall_score(y_test_encoded, y_pred, pos_label=0)
                auc_roc = roc_auc_score(y_test_encoded, y_pred)
                mcc = matthews_corrcoef(y_test_encoded, y_pred)
                precision = precision_score(y_test_encoded, y_pred, pos_label=1)
                f1 = f1_score(y_test_encoded, y_pred, pos_label=1)
                # Append metrics to lists
                accuracy_list.append(accuracy)
                sensitivity_list.append(sensitivity)
                specificity_list.append(specificity)
                auc_roc_list.append(auc_roc)
                mcc_list.append(mcc)
                precision_list.append(precision)
                f1_list.append(f1)

                # Print metrics for the current fold
                print(f"Accuracy: {accuracy}")
                print(f"Sensitivity: {sensitivity}")
                print(f"Specificity: {specificity}")
                print(f"AUC-ROC: {auc_roc}")
                print(f"MCC: {mcc}")
                print(f"Precision: {precision}")
                print(f"F1 Score: {f1}")
                gc.collect()

            # Calculate average metrics
            average_accuracy = np.mean(accuracy_list)
            average_sensitivity = np.mean(sensitivity_list)
            average_specificity = np.mean(specificity_list)
            average_auc_roc = np.mean(auc_roc_list)
            average_mcc = np.mean(mcc_list)
            average_precision = np.mean(precision_list)
            average_f1 = np.mean(f1_list)

            time_end = time.time()
            memory_end = psutil.Process(os.getpid()).memory_info().rss / 1024 ** 2
            del X_train, X_test, y_train, y_test
            gc.collect()
            tf.keras.backend.clear_session()
    else:
        for fold, (train_index, test_index) in enumerate(skf.split(data, labels)):
            print(f'\n[INFO] Fold {fold + 1} / 10 for {name}')

            X_train, X_test = data[train_index], data[test_index]
            y_train, y_test = labels[train_index], labels[test_index]

            # Train the model
            model.fit(X_train, y_train)

            # Evaluate the model
            predictions = model.predict(X_test)
            y_pred = predictions
            y_test_encoded = y_test

            # Calculate evaluation metrics for the current fold
            accuracy = np.mean(y_pred == y_test_encoded)
            sensitivity = recall_score(y_test, y_pred, pos_label=1)
            specificity = recall_score(y_test, y_pred, pos_label=0)
            auc_roc = roc_auc_score(y_test, predictions)
            mcc = matthews_corrcoef(y_test, y_pred)
            precision = precision_score(y_test, y_pred, pos_label=1)
            f1 = f1_score(y_test, y_pred, pos_label=1)
            # Append metrics to lists
            accuracy_list.append(accuracy)
            sensitivity_list.append(sensitivity)
            specificity_list.append(specificity)
            auc_roc_list.append(auc_roc)
            mcc_list.append(mcc)
            precision_list.append(precision)
            f1_list.append(f1)

            # Print metrics for the current fold
            print(f"Accuracy: {accuracy}")
            print(f"Sensitivity: {sensitivity}")
            print(f"Specificity: {specificity}")
            print(f"AUC-ROC: {auc_roc}")
            print(f"MCC: {mcc}")
            print(f"Precision: {precision}")
            print(f"F1 Score: {f1}")
            gc.collect()

        # Calculate average metrics
        average_accuracy = np.mean(accuracy_list)
        average_sensitivity = np.mean(sensitivity_list)
        average_specificity = np.mean(specificity_list)
        average_auc_roc = np.mean(auc_roc_list)
        average_mcc = np.mean(mcc_list)
        average_precision = np.mean(precision_list)
        average_f1 = np.mean(f1_list)

        time_end = time.time()
        memory_end = psutil.Process(os.getpid()).memory_info().rss / 1024 ** 2
        

    # Append results to DataFrame
    results_df.loc[len(results_df)] = [name, average_accuracy, average_sensitivity, average_specificity,
                                       average_auc_roc, average_mcc, average_precision, average_f1,
                                       memory_end - memory_start, time_end - time_start]





In [11]:
from sklearn.linear_model import LogisticRegression
# SVM
lr_model = LogisticRegression(n_jobs=-1, random_state=0)
evaluate_model(lr_model, 'LR', data, labels)




[INFO] Fold 1 / 10 for LR


/opt/tljh/user/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy: 0.8621997471554994
Sensitivity: 0.8968692449355433
Specificity: 0.7862903225806451
AUC-ROC: 0.8415797837580942
MCC: 0.6809483462308427
Precision: 0.9018518518518519
F1 Score: 0.899353647276085

[INFO] Fold 2 / 10 for LR


/opt/tljh/user/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy: 0.8647281921618205
Sensitivity: 0.9060773480662984
Specificity: 0.7741935483870968
AUC-ROC: 0.8401354482266975
MCC: 0.6840915569683126
Precision: 0.8978102189781022
F1 Score: 0.9019248395967003

[INFO] Fold 3 / 10 for LR


...

Accuracy: 0.8596713021491783
Sensitivity: 0.9005524861878453
Specificity: 0.7701612903225806
AUC-ROC: 0.8353568882552129
MCC: 0.6729512722907667
Precision: 0.8956043956043956
F1 Score: 0.8980716253443527

[INFO] Fold 4 / 10 for LR


/opt/tljh/user/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy: 0.8445006321112516
Sensitivity: 0.8839779005524862
Specificity: 0.7580645161290323
AUC-ROC: 0.8210212083407591
MCC: 0.6399642816082987
Precision: 0.8888888888888888
F1 Score: 0.8864265927977838

[INFO] Fold 5 / 10 for LR


/opt/tljh/user/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy: 0.8735777496839444
Sensitivity: 0.9208103130755064
Specificity: 0.7701612903225806
AUC-ROC: 0.8454858016990434
MCC: 0.7023448926105983
Precision: 0.8976660682226212
F1 Score: 0.909090909090909

[INFO] Fold 6 / 10 for LR
Accuracy: 0.8558786346396966
Sensitivity: 0.9134438305709024
Specificity: 0.7298387096774194
AUC-ROC: 0.8216412701241609
MCC: 0.6588793528450586
Precision: 0.8809946714031972
F1 Score: 0.8969258589511755

[INFO] Fold 7 / 10 for LR
Accuracy: 0.865992414664981
Sensitivity: 0.9208103130755064
Specificity: 0.7459677419354839
AUC-ROC: 0.8333890275054952
MCC: 0.682944531973988
Precision: 0.8880994671403197
F1 Score: 0.9041591320072333

[INFO] Fold 8 / 10 for LR
Accuracy: 0.8571428571428571
Sensitivity: 0.9097605893186004
Specificity: 0.7419354838709677
AUC-ROC: 0.825848036594784
MCC: 0.6632477714110754
Precision: 0.8853046594982079
F1 Score: 0.8973660308810174

[INFO] Fold 9 / 10 for LR
Accuracy: 0.8445006321112516
Sensitivity: 0.9042357274401474
Specificity: 0.7137

In [12]:
#NB
nb_model = GaussianNB()
evaluate_model(nb_model, 'NB', data, labels)


[INFO] Fold 1 / 10 for NB
Accuracy: 0.7092288242730721
Sensitivity: 0.7716390423572744
Specificity: 0.5725806451612904
AUC-ROC: 0.6721098437592823
MCC: 0.3380189055195261
Precision: 0.7980952380952381
F1 Score: 0.7846441947565543

[INFO] Fold 2 / 10 for NB
Accuracy: 0.7395701643489254
Sensitivity: 0.7826887661141805
Specificity: 0.6451612903225806
AUC-ROC: 0.7139250282183806
MCC: 0.415753839185266
Precision: 0.8284600389863548
F1 Score: 0.8049242424242425

[INFO] Fold 3 / 10 for NB
Accuracy: 0.7256637168141593
Sensitivity: 0.7955801104972375
Specificity: 0.5725806451612904
AUC-ROC: 0.6840803778292639
MCC: 0.366194527246659
Precision: 0.8029739776951673
F1 Score: 0.799259944495837

[INFO] Fold 4 / 10 for NB
Accuracy: 0.7206068268015171
Sensitivity: 0.8066298342541437
Specificity: 0.532258064516129
AUC-ROC: 0.6694439493851363
MCC: 0.3432043319299735
Precision: 0.7906137184115524
F1 Score: 0.7985414767547858

[INFO] Fold 5 / 10 for NB
Accuracy: 0.7787610619469026
Sensitivity: 0.825046040

In [13]:

# KNN
knn_model = KNeighborsClassifier()
evaluate_model(knn_model, 'KNN', data, labels)




[INFO] Fold 1 / 10 for KNN
Accuracy: 0.7547408343868521
Sensitivity: 0.9281767955801105
Specificity: 0.375
AUC-ROC: 0.6515883977900552
MCC: 0.3772176808561706
Precision: 0.7647951441578149
F1 Score: 0.8386023294509152

[INFO] Fold 2 / 10 for KNN
Accuracy: 0.7686472819216182
Sensitivity: 0.9226519337016574
Specificity: 0.4314516129032258
AUC-ROC: 0.6770517733024416
MCC: 0.4201413083433931
Precision: 0.780373831775701
F1 Score: 0.8455696202531646

[INFO] Fold 3 / 10 for KNN
Accuracy: 0.7724399494310998
Sensitivity: 0.9263351749539595
Specificity: 0.43548387096774194
AUC-ROC: 0.6809095229608506
MCC: 0.43040849570816625
Precision: 0.7822706065318819
F1 Score: 0.8482293423271501

[INFO] Fold 4 / 10 for KNN
Accuracy: 0.7560050568900126
Sensitivity: 0.9263351749539595
Specificity: 0.38306451612903225
AUC-ROC: 0.6546998455414959
MCC: 0.38152836112826777
Precision: 0.7667682926829268
F1 Score: 0.8390325271059216

[INFO] Fold 5 / 10 for KNN
Accuracy: 0.7446270543615676
Sensitivity: 0.8987108655

In [14]:
# Random Forest
rf_model = RandomForestClassifier(random_state=0, n_jobs=-1)
evaluate_model(rf_model, 'Random Forest', data, labels)



[INFO] Fold 1 / 10 for Random Forest
Accuracy: 0.8015170670037927
Sensitivity: 0.9465930018416207
Specificity: 0.4838709677419355
AUC-ROC: 0.7152319847917781
MCC: 0.5107424003785218
Precision: 0.8006230529595015
F1 Score: 0.8675105485232067

[INFO] Fold 2 / 10 for Random Forest
Accuracy: 0.8053097345132744
Sensitivity: 0.9373848987108656
Specificity: 0.5161290322580645
AUC-ROC: 0.726756965484465
MCC: 0.5213550173483098
Precision: 0.809220985691574
F1 Score: 0.8686006825938568

[INFO] Fold 3 / 10 for Random Forest
Accuracy: 0.8217446270543616
Sensitivity: 0.9429097605893186
Specificity: 0.5564516129032258
AUC-ROC: 0.7496806867462722
MCC: 0.5652000042007828
Precision: 0.8231511254019293
F1 Score: 0.878969957081545

[INFO] Fold 4 / 10 for Random Forest
Accuracy: 0.7926675094816688
Sensitivity: 0.9392265193370166
Specificity: 0.4717741935483871
AUC-ROC: 0.7055003564427019
MCC: 0.48640013419687045
Precision: 0.7956318252730109
F1 Score: 0.8614864864864864

[INFO] Fold 5 / 10 for Random For

In [15]:
import lightgbm
# LGBM
lgb_model = lightgbm.LGBMClassifier(random_state=0, n_jobs = -1)
evaluate_model(lgb_model, 'LGBM', data, labels)


[INFO] Fold 1 / 10 for LGBM
[LightGBM] [Info] Number of positive: 4886, number of negative: 2232
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 92.874668 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3102509
[LightGBM] [Info] Number of data points in the train set: 7118, number of used features: 23642
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.686429 -> initscore=0.783476
[LightGBM] [Info] Start training from score 0.783476
Accuracy: 0.8470290771175727
Sensitivity: 0.9373848987108656
Specificity: 0.6491935483870968
AUC-ROC: 0.7932892235489811
MCC: 0.6314099496183003
Precision: 0.8540268456375839
F1 Score: 0.893766461808604

[INFO] Fold 2 / 10 for LGBM
[LightGBM] [Info] Number of positive: 4886, number of negative: 2232
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 777.991539 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM]

In [16]:
# Save results to a CSV file
results_df.to_csv('DS3_VGG16.csv', index=False)

In [17]:
results_df

,Model,Average Accuracy,Average Sensitivity,Average Specificity,Average AUC-ROC,Average MCC,Average Precision,Average F1 Score,Memory Used (MB),Time (s)
0,LR,0.858136,0.906429,0.752419,0.829424,0.667127,0.889161,0.897655,-435.050781,770.336060
1,NB,0.742068,0.805679,0.602823,0.704251,0.405765,0.816369,0.810848,681.558594,38.406950
2,KNN,0.768618,0.926873,0.422177,0.674525,0.419373,0.778369,0.846131,692.359375,66.133160
3,Random Forest,0.813631,0.946398,0.522984,0.734691,0.543672,0.812969,0.874576,635.347656,131.472905
4,LGBM,0.846632,0.934794,0.653629,0.794211,0.630873,0.855420,0.893273,640.531250,19033.673472
